# 01 Data Loading
---
This notebook pulls the data from the excel sheet and contains functions that allow for getting selected data in selected timeframes

---

In [6]:
#importing libraries
import pandas as pd
from datetime import datetime, timedelta
import numpy as np

from sklearn.preprocessing import MinMaxScaler
import importlib
import config
importlib.reload(config)

#loading data
all_data = pd.read_excel('data/all_data_raw.xlsx', sheet_name=None)

In [7]:
def get_data(sheet_name, start_date, end_date, frequency = "D"):

    """
    Function to return any sheet from the excel with specified
    start & end date. Frequency can be specified, daily by default
    Date input format: "YYYY-MM-DD"

    example:
    get_data("GDP_CURY_US_GDP_GROWTH", "2010-01-01", "2019-12-31","Q")
    """
    
    #get selected sheet and filter by time period
    data = all_data[sheet_name].iloc[:,0:2]

    #check if timeframe is valid and within range of data
    start_date = datetime.strptime(start_date, "%Y-%m-%d")
    end_date = datetime.strptime(end_date, "%Y-%m-%d")
    if start_date > end_date:
        raise ValueError("Start date must be before end date")
    elif start_date < data["Date"].min():
        raise ValueError("Start date is before start of data")
    elif end_date > data["Date"].max():
        raise ValueError("End date is after end of data")

    #filter data by timeframe
    data = data[data["Date"] >= start_date]
    data = data[data["Date"] <= end_date]
    
    #set date as index, resample and drop na
    data = data.set_index("Date")
    data = data.resample(frequency).ffill()
    data = data.iloc[::-1]
    return data

In [ ]:
def get_indicators(start_date, end_date):

    """
    Function to return all marcoeconomic indicators for a given timeframe.
    If start_date and end_date are the same, only one day is returned.
    Date input format: "YYYY-MM-DD"

    example:
    get_indicators("2013-01-01","2013-12-31")
    """
    #creating empty dataframe
    indicators = pd.DataFrame()
    day = start_date

    #in case of timeframe vs case of one day
    if start_date == end_date:
       
        #converting day to datetime
        day_datetime = datetime.strptime(day, '%Y-%m-%d')

        #looping through all indicators
        for key in config.PARAM_DICT:
            temp_day = day

            #if no data for the day, go back one day (holiday, weekend)
            while get_data(key,temp_day,temp_day,"D").empty:
                temp_day = datetime.strptime(temp_day,'%Y-%m-%d') - timedelta(days=1)
                temp_day = temp_day.strftime("%Y-%m-%d")

            #getting data for the day and making sure it's adding to dataframe properly
            data_to_add = get_data(key, temp_day, end_date)
            data_to_add.index = [day_datetime]
            data_to_add.columns.values[0] = key
            indicators = pd.concat([indicators, data_to_add], axis = 1)
    else:
        for key in config.PARAM_DICT:
            temp_day = day
            day_datetime = datetime.strptime(day, '%Y-%m-%d')

            if get_data(key,temp_day,temp_day,"D").empty:
                #if no data for the day, go back one day (holiday, weekend)
                while get_data(key,temp_day,temp_day,"D").empty:
                    temp_day = datetime.strptime(temp_day,'%Y-%m-%d') - timedelta(days=1)
                    temp_day = temp_day.strftime("%Y-%m-%d")
                
                data_to_add = get_data(key,temp_day, end_date)
                data_to_add.columns.values[0] = key
                indicators = pd.concat([indicators,data_to_add], axis = 1)

            else:
                data_to_add = get_data(key,temp_day,end_date)
                data_to_add.columns.values[0] = key
                indicators = pd.concat([indicators,data_to_add], axis = 1)
        
    indicators = indicators.ffill()
    indicators = indicators.loc[start_date:end_date]

    colnames = indicators.columns
    index = indicators.index

    scaler = MinMaxScaler()
    scaled_data = pd.DataFrame(scaler.fit_transform(indicators))
    scaled_data.columns = colnames
    scaled_data.index = index
    return scaled_data

In [13]:
def get_indices(start_date, end_date,freq = "D"):
    """
    Function to return all index prices for a given timeframe. 
    Date input format: "YYYY-MM-DD"

    frequency is set to day, can be adjusted to "m", "q" or "y"

    example:
    get_indices("2012-01-01", "2012-12-31")
    """
    date_range = pd.date_range(start_date, end_date)
    df_prices = pd.DataFrame(index=date_range)
    
    for key in config.ASSET_DICT:

        data_to_add = get_data(key,start_date,end_date,freq)
        data_to_add.columns.values[0] = key
        df_prices = pd.concat([df_prices,data_to_add], axis = 1)
        
    #add another column "cash" and fill it up with 0
    df_prices["cash"] = 0.00001
    return df_prices